In [82]:
import matplotlib.pyplot as plt
from shapely import LineString
import segyio
from shapely import Point
import numpy as np
import plotly.graph_objects as go
from shapely.geometry import Polygon

In [20]:
def read_seismic_cube(file_path):
   """
   :param file_path: Путь к файлу
   :return: возвращает куб в виде списков по Inlines, Xlines и Sampels (координаты x, y, z)
   а также словарю сейсмотрасс, где ключ это координата пересечения Inlines и Xlines, а значение это список значений сейсмотрасс
   """
   with segyio.open(file_path, 'r') as segyfile:
       x = list(segyfile.ilines)
       y = list(segyfile.xlines)
       z = list(segyfile.samples)
       cube = segyio.tools.cube(segyfile)
       traces = {f"{x[i]}_{y[j]}": cube[i][j][:] for i in range(len(x)) for j in range(len(y))}
   return x, y, z, traces

In [21]:
file_path = 'C:/HV/Seismic/datas/Cube_TWT.segy'
ilines, xlines, samples, traces = read_seismic_cube(file_path)

In [22]:
def extract_segy_data(file_path):
    with segyio.open(file_path, 'r') as segyfile:
        # Извлечение координат линий и образцов
        x = list(segyfile.ilines)
        y = list(segyfile.xlines)
        z = list(segyfile.samples)
        coord_x = set()
        coord_y = set()
       
        # Создание куба данных
        cube = segyio.tools.cube(segyfile)
        
        # Извлечение координат из заголовков
        traces = {}
        for i in range(len(x)):
            for j in range(len(y)):
                # Получение индекса записи
                index = i * len(y) + j
                
                # Получение данных записи
                trace_data = cube[i][j][:]
                
                # Извлечение заголовка записи
                trace_header = segyfile.header[index]
                
                # Получение координат X и Y из заголовка записи
                coord_x.add(trace_header[segyio.TraceField.SourceX])
                coord_y.add(trace_header[segyio.TraceField.SourceY])
                
        
    return coord_x, coord_y

In [23]:
file_path = 'C:/HV/Seismic/datas/Cube_TWT.segy'
coord_x, coord_y = extract_segy_data(file_path)

In [72]:
print(sorted(samples))

[2142.0, 2144.0, 2146.0, 2148.0, 2150.0, 2152.0, 2154.0, 2156.0, 2158.0, 2160.0, 2162.0, 2164.0, 2166.0, 2168.0, 2170.0, 2172.0, 2174.0, 2176.0, 2178.0, 2180.0, 2182.0, 2184.0, 2186.0, 2188.0, 2190.0, 2192.0, 2194.0, 2196.0, 2198.0, 2200.0, 2202.0, 2204.0, 2206.0, 2208.0, 2210.0, 2212.0, 2214.0, 2216.0, 2218.0, 2220.0, 2222.0, 2224.0, 2226.0, 2228.0, 2230.0, 2232.0, 2234.0, 2236.0, 2238.0, 2240.0, 2242.0, 2244.0, 2246.0, 2248.0, 2250.0, 2252.0, 2254.0, 2256.0, 2258.0, 2260.0, 2262.0, 2264.0, 2266.0, 2268.0, 2270.0, 2272.0, 2274.0, 2276.0, 2278.0, 2280.0, 2282.0, 2284.0, 2286.0, 2288.0, 2290.0, 2292.0, 2294.0, 2296.0, 2298.0, 2300.0, 2302.0, 2304.0, 2306.0, 2308.0, 2310.0, 2312.0, 2314.0, 2316.0, 2318.0, 2320.0, 2322.0, 2324.0, 2326.0, 2328.0, 2330.0, 2332.0, 2334.0, 2336.0, 2338.0, 2340.0, 2342.0, 2344.0, 2346.0, 2348.0, 2350.0, 2352.0, 2354.0, 2356.0, 2358.0, 2360.0, 2362.0, 2364.0, 2366.0, 2368.0, 2370.0, 2372.0, 2374.0, 2376.0, 2378.0, 2380.0, 2382.0, 2384.0, 2386.0]


In [65]:
# Перевод множества в упорядоченный список в порядке убывания
sorted_coord_x = sorted(coord_x, reverse=False)
sorted_coord_y = sorted(coord_y, reverse=False)
sorted_coord_z = sorted(samples, reverse=False)
poligon_cub_verh = [(sorted_coord_x[0], sorted_coord_y[0]), (sorted_coord_x[0], sorted_coord_y[-1]), (sorted_coord_x[-1], sorted_coord_y[-1]),(sorted_coord_x[-1], sorted_coord_y[0]), (sorted_coord_x[0], sorted_coord_y[0])]
poligon_cub_sboky = [(sorted_coord_x[0], -1* sorted_coord_z[-1]), (sorted_coord_x[0], -1*sorted_coord_z[0]), (sorted_coord_x[-1], -1*sorted_coord_z[0]),(sorted_coord_x[-1], -1*sorted_coord_z[-1]), (sorted_coord_x[0], -1* sorted_coord_z[-1])]

In [26]:
def process_file(file_path):
    # Множества для хранения значений
    x_set = set()
    y_set = set()
    z_set = set()
    
    with open(file_path, 'r') as file:
        for line in file:
            # Пропуск строк, начинающихся с "#"
            if line.startswith("#"):
                continue
            
            # Разделение строки на колонки
            columns = line.split()
            if len(columns) >= 5:
                # Получение значений из колонок
                x_value = columns[0]
                y_value = columns[1]
                z_value = columns[2]
                
                # Обработка значений
                x_set.add(int(x_value.split('.')[0]))
                y_set.add(int(y_value.split('.')[0]))
                z_set.add(int(z_value.lstrip('-').split('.')[0]))
    
    return x_set, y_set, z_set

In [58]:
def process_file_2(file_path):
    # Множества для хранения значений
    x_set = []
    y_set = []
    z_set = []
    
    with open(file_path, 'r') as file:
        for line in file:
            # Пропуск строк, начинающихся с "#"
            if line.startswith("#"):
                continue
            
            # Разделение строки на колонки
            columns = line.split()
            if len(columns) >= 5:
                # Получение значений из колонок
                x_value = columns[0]
                y_value = columns[1]
                z_value = columns[2]
                
                # Обработка значений
                x_set.append(int(x_value.split('.')[0]))
                y_set.append(int(y_value.split('.')[0]))
                z_set.append(int(z_value.lstrip('-').split('.')[0]))
    
    return x_set, y_set, z_set

In [95]:
file_path = 'C:/HV/Seismic/datas/TWT_Top_U1.txt'  # Укажите путь к вашему файлу
x_set, y_set, z_set = process_file_2(file_path)

In [93]:
def show_grafic(polygon_coordinates, x, y, val):
    x_values = list(sorted(set(x)))
    y_values = list(sorted(set(y)))
    values = np.array(val).reshape((len(x_values), len(y_values)))
    
    # Разделение координат полигона на отдельные списки
    x_polygon, y_polygon = zip(*polygon_coordinates)
    
    # Создание полигона
    polygon = Polygon(polygon_coordinates)

    # Определение границ тепловой карты
    heatmap_bounds = Polygon([(x_values[0], y_values[0]), (x_values[0], y_values[-1]),
                          (x_values[-1], y_values[-1]), (x_values[-1], y_values[0])])

    #    Вычисление пересечения полигона и границ тепловой карты
    intersection = polygon.intersection(heatmap_bounds)

    # Вычисление площадей
    polygon_area = polygon.area
    heatmap_area = heatmap_bounds.area
    intersection_area = intersection.area

    # Вычисление процента перекрытия
    if polygon_area > 0:
        overlap_percentage = (intersection_area / heatmap_area) * 100
    else:
        overlap_percentage = 0

    print(f"Процент перекрытия тепловой карты от полигона: {overlap_percentage:.2f}%")
    
        
    # Создание полигона
    polygon = go.Scatter(
        x=list(x_polygon) + [x_polygon[0]],  # Замыкаем полигон
        y=list(y_polygon) + [y_polygon[0]],  # Замыкаем полигон
        fill='toself',
        fillcolor='rgba(0, 100, 200, 0.1)',  # Полупрозрачный цвет
        line=dict(color='blue'),
        name='Polygon'
    )

    # Создание тепловой карты
    heatmap = go.Heatmap(
        z=values,  # Значения в виде двумерного массива
        x=x_values,
        y=y_values,
        colorscale='Viridis',
        colorbar=dict(title='Values')
    )

    # Объединение графиков
    fig = go.Figure(data=[heatmap, polygon])

    # Настройка макета
    fig.update_layout(
        title='Границы куба и горизонт. Вид сверху',
        xaxis_title='Инлайны',
        yaxis_title='Икслайны',
        height=600,
        width=600
    )

    # Отображение графика
    fig.show()
    

In [96]:
show_grafic(poligon_cub_verh, x_set, y_set, z_set)

Процент перекрытия тепловой карты от полигона: 100.00%


In [91]:
def show_grafic_2(polygon_coordinates, x, y, val):
    x_values = list(sorted(set(x)))
    y_values = list(sorted(set(y)))
    values = np.array(val).reshape((len(y_values)),len(x_values))
    
    # Создание полигона
    polygon = Polygon(polygon_coordinates)

    # Определение границ тепловой карты
    heatmap_bounds = Polygon([(x_values[0], y_values[0]), (x_values[0], y_values[-1]),
                          (x_values[-1], y_values[-1]), (x_values[-1], y_values[0])])

    #    Вычисление пересечения полигона и границ тепловой карты
    intersection = polygon.intersection(heatmap_bounds)

    # Вычисление площадей
    polygon_area = polygon.area
    heatmap_area = heatmap_bounds.area
    intersection_area = intersection.area

    # Вычисление процента перекрытия
    if polygon_area > 0:
        overlap_percentage = (intersection_area / heatmap_area) * 100
    else:
        overlap_percentage = 0

    print(f"Процент перекрытия тепловой карты от полигона: {overlap_percentage:.2f}%")

    # Создание полигона для графика
    x_polygon, y_polygon = zip(*polygon_coordinates)


    # Создание полигона
    polygon = go.Scatter(
        x=list(x_polygon) + [x_polygon[0]],  # Замыкаем полигон
        y=list(y_polygon) + [y_polygon[0]],  # Замыкаем полигон
        fill='toself',
        fillcolor='rgba(0, 100, 200, 0.1)',  # Полупрозрачный цвет
        line=dict(color='blue'),
        name='Polygon'
    )

    # Создание тепловой карты
    heatmap = go.Heatmap(
        z=values,  # Значения в виде двумерного массива
        x=x_values,
        y=y_values,
        colorscale='Viridis',
        colorbar=dict(title='Values')
    )

    # Объединение графиков
    fig = go.Figure(data=[heatmap, polygon])

    # Настройка макета
    fig.update_layout(
        title='Границы куба и горизонт. Вид сбоку',
        xaxis_title='Инлайны',
        yaxis_title='Глубины',
        height=600,
        width=600
    )

    # Отображение графика
    fig.show()
    

In [97]:
yy = []
for _ in range(len(set(x_set))*len(set(z_set))):
    yy.append(1)
zz = []
for k in z_set:
    zz.append(-1*k)
show_grafic_2(poligon_cub_sboky, x_set, zz, yy)

Процент перекрытия тепловой карты от полигона: 100.00%
